In [1]:
import os
os.environ['HF_HOME'] = '/mnt/opr/levlevi/tmp'

import pandas as pd
from tqdm import tqdm
from glob import glob
from mini_cpm_full_infer import load_and_convert_image, ocr, load_model_and_tokenizer

test_set_dir = '/mnt/opr/levlevi/player-re-id/src/testing/ocr_model_comparions/nba_100_test_set'
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))

model = load_model_and_tokenizer(2)
def perform_ocr(image_file_path: str) -> str:
    img = load_and_convert_image(image_file_path)
    result = ocr(img, model)
    return result

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

/playpen-storage/levlevi/Anaconda3/envs/tg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-23 03:38:01,106 - INFO - Loading model and tokenizer...
2024-06-23 03:38:01,289 - INFO - vision_config is None, using default vision config
Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.35it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-23 03:38:17,914 - INFO - Model and tokenizer loaded successfully.
100%|██████████| 100/100 [02:36<00:00,  1.57s/it]


In [10]:
import ast
from tqdm import tqdm

results_clone = results.copy()
results_parsed = []
for idx, result in tqdm(enumerate(results_clone)):
    if idx == 100:
        break
    if not result:
        results_parsed.append('na')
        continue
    if '{' not in result:
        results_parsed.append('na')
        continue
    result_str = '{' + result.strip().split('{')[-1]
    result_str = result_str.split('}')[0] + '}'
    result_dict = ast.literal_eval(result_str)
    jersey_number = result_dict['jersey_number']
    if jersey_number is None:
        results_parsed.append('na')
        continue
    results_parsed.append(jersey_number)

100it [00:00, 43062.67it/s]


In [11]:
results_df = pd.DataFrame()
results_df['ground_truth'] = ground_truth_labels
results_df['predicted'] = results_parsed
results_df['correct'] = results_df['ground_truth'] == results_df['predicted']
results_df

,ground_truth,predicted,correct
0,na,na,True
1,na,na,True
2,na,na,True
3,2,2,True
4,36,34,False
...,...,...,...
95,41,41,True
96,13,na,False
97,na,na,True
98,2,12,False


In [12]:
sum(results_df['correct'])

65